<font size="5">
 <div class="alert alert-block alert-info"><b>Master in Data Science - Iscte <b>
     </div>
</font> 
 
 
     
    
  <font size="5"> OEOD - Week 5 </font>
  
  
  
  <font size="3"> **Diana Aldea Mendes**, November 2024 </font>
  
   
  <font size="3"> *diana.mendes@iscte-iul.pt* </font> 
  
    
 
  
    
  <font color='blue'><font size="5"> <b>Week 5 - Case study 1 - RL for finance (algorithmic trading)<b></font></font>

# RL for algorithmic trading

- **Main purpose**: define a trading strategy and predict if you buy or sell stocks

- A Trading Strategy is based on predefined rules used to make the trading decisions.
- The simplest trading strategy is the Moving Average (MA) (Simple MA (SMA) and Exponential MA (EMA)): 
    - The SMA calculates the average closing price over a set time period, while the EMA puts more weight on recent prices.
    - A moving average crossover trading strategy involves buying when a shorter-term MA crosses above a longer-term MA, and selling when it crosses below.
    - The crossover indicates a potential trend change. 
    - For example, if the 50-day MA crosses above the 200-day MA, it's a buy signal as the short-term trend is now up.
    

- Relative strength index (RSI) is a powerful momentum indicator that measures the magnitude of recent price changes to analyze overbought or oversold conditions. 
    - The RSI oscillates between 0 and 100. 
    - A reading under 30 is considered oversold and above 70 is overbought. 
    - The basic RSI trading strategies are:
        - Buy when the RSI drops below 30
        - Sell when it rises above 70

In [ ]:
# import libraries

import yfinance as yf  # import data from web (financial time series)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  ## interactive data visualization

# high-quality figures
%config InlineBackend.figure_format = 'svg'


## Import and visualize data

In [ ]:

# historical stock price data for QQQ , from the period 2016 to 2024
## QQQ - ticker for the biggest online retail companies (Amazon, Booking, Costco, Airbnb, eBay, etc)

qqq = yf.Ticker("QQQ")
data = qqq.history(start="2016-01-01", end="2024-10-31")


data.head()


In [ ]:
data.Close.plot();

In [ ]:
## visualize signal and moving average lines

start_date = '2021-01-01'
end_date = '2022-12-31'

# Calculating the short-window simple moving average
short_rolling = data.rolling(window=20).mean()
short_rolling.head(20)

# Calculating the long-window simple moving average
long_rolling = data.rolling(window=100).mean()
long_rolling.tail()


fig, ax = plt.subplots(figsize=(8,5))

ax.plot(data.loc[start_date:end_date, :].index, data.loc[start_date:end_date, 'Close'], label='QQQ Close Price')
ax.plot(long_rolling.loc[start_date:end_date, :].index, long_rolling.loc[start_date:end_date, 'Close'], label = '100-days SMA')
ax.plot(short_rolling.loc[start_date:end_date, :].index, short_rolling.loc[start_date:end_date, 'Close'], label = '20-days SMA')

ax.legend(loc='best')
ax.set_ylabel('Price in $')


## Define trading strategy and backtesting

- Define a trading strategy: 
    - Buy when the price is above the 50-day moving average 
    - sell otherwise

- To define a *moving average* (that is, smooth the data), we use the 'rolling' function and the window size (10 days, 20 days, 50 days, etc). We compute the mean of the last x values and roll this operations until the end of data. The resulting signal is smoother that the original data (highlight the trend and decrease the volatility)

- When the price time series (signal) crosses the MA time series from below, we will close any existing short position and go long (buy) one unit of the asset.
- When the price time series crosses the MA time series  from above, we will close any existing long position and go short (sell) one unit of the asset.

`` data.rolling(window=20).mean()``

- Backtesting: estimate (define some metrics, measure) the performance of the strategy
- We need:
    - log-returns (pct_change() function)
    - cumulative log-returns
    - variance - volatility
    - Sharpe-ratio: measure the risk adjusted performance of an investment over time
    - Value at Risk

In [ ]:
# Define the trading strategy
def trading_strategy(data):
    # Define your trading strategy based on the historical data
    # Example: Buy when the price is above the 50-day moving average, sell otherwise
    signals = pd.DataFrame(index=data.index)
    signals['Signal'] = np.where(data['Close'] > data['Close'].rolling(window=50).mean(), 1, 0)
    return signals

In [ ]:
# Perform backtesting
def backtest(data, signals):
    # Combine the historical data with the trading signals
    df = pd.concat([data, signals], axis=1).dropna()

    # Calculate daily returns
    df['Return'] = df['Close'].pct_change()

    # Calculate cumulative returns
    df['Cumulative Return'] = (1 + df['Return']).cumprod()

    # Calculate portfolio value
    df['Portfolio Value'] = df['Cumulative Return'] * initial_investment

    # Calculate risk-free rate (assumed to be 0 in this example)
    risk_free_rate = 0

    # Calculate metrics
    num_trading_days = len(df)
    returns = df['Return']
    cumulative_returns = df['Cumulative Return']
    portfolio_value = df['Portfolio Value']
    annual_returns = (cumulative_returns[-1]) ** (252/num_trading_days) - 1
    volatility = returns.std() * np.sqrt(252)  # Annualized volatility
    sharpe_ratio = (annual_returns - risk_free_rate) / volatility
    cagr = (cumulative_returns[-1]) ** (252/num_trading_days) - 1
    # alpha, beta = np.polyfit(returns - risk_free_rate, market_returns - risk_free_rate, deg=1)
    variance = returns.var() * 252
    cvar = returns[returns <= np.percentile(returns, 5)].mean() * 252

    return df, sharpe_ratio, cagr, cumulative_returns,  variance, cvar

## Apply defined rules to data

In [ ]:
# Define initial investment amount
initial_investment = 10000

# Perform the trading strategy
signals = trading_strategy(data)

# Perform backtesting and calculate metrics
backtest_results, sharpe_ratio, cagr, cumulative_returns,  variance, cvar = backtest(data, signals)

In [ ]:
# Plot the cumulative returns with entry and exit points
plt.figure(figsize=(10, 6))
plt.plot(backtest_results.index, cumulative_returns, label='Cumulative Returns')
plt.scatter(backtest_results[backtest_results['Signal'] == 1].index, backtest_results[backtest_results['Signal'] == 1]['Cumulative Return'], color='green', s=8, label='Buy')
plt.scatter(backtest_results[backtest_results['Signal'] == 0].index, backtest_results[backtest_results['Signal'] == 0]['Cumulative Return'], color='red',  s=8, label='Sell')
plt.title('Cumulative Returns with Entry and Exit Points')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Print the calculated metrics
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"CAGR: {cagr:.2%}")
print(f"Cumulative Returns: {cumulative_returns[-1]:.2%}")
#print(f"Alpha: {alpha:.4f}")
#print(f"Beta: {beta:.4f}")
print(f"Variance: {variance:.6f}")
print(f"CVaR: {cvar:.6f}")

####################################################################

- CAGR - Compound Annual Growth Rate - measures the average annual growth rate of an investment over a given period of time. 
- For example, if you invest 1000 dollars in a stock that has a 10% CAGR, your investment will grow to 1100 dollars after one year, 1210 dollars after two years, and so on.
- The Sharpe ratio is a measure of risk-adjusted return. 
- For example, if an investment has a 10% return and a standard deviation of 5%, the Sharpe ratio would be 2. This means that the investment has a higher return than the benchmark, but with a lower level of risk. The Sharpe ratio is calculated as (R_a-R_b)/σ_a, where R_a is the expected return on the asset, R_b is the return on the risk-free asset, and σ_a is the standard deviation of the asset’s returns


## Q-learning agent

- Environment: the market including historical price data, relevant indicators, and other market factors.
- State: the current market condition of the environment such as price, volume, or market information at a particular time.
- Action: the trading action such as “buy” or “sell”.
- Policy: the objective - such as maximizing returns, minimizing risk, or achieving a specific target.
- Reward: This is the gains in the market.
- The reinforcement learning agent: Develop an agent that learns to make trading decisions based on the observed environment state (Q-learning). 

- Epsilon-greedy policy is a RL algorithm that uses a greedy approach to make decisions. It works by choosing the action that has the highest expected reward.
-  Q-learning update rule: multiply the current Q-value by a learning rate, and add a negative exponential decay term. 

- **Note**: we can incorporate human feedback in reinforcement learning for algorithmic trading - ongoing research area - **RLHF** (*reinforcement learning with human feedback*)

In [ ]:
# Define the Q-learning agent
class QLearningAgent:
    def __init__(self, num_states, num_actions, alpha, gamma):
        self.num_states = num_states
        self.num_actions = num_actions
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.q_table = np.zeros((num_states, num_actions))

    def update_q_table(self, state, action, reward, next_state):
        # Q-learning update rule
        max_q_value = np.max(self.q_table[next_state])
        self.q_table[state, action] += self.alpha * (reward + self.gamma * max_q_value - self.q_table[state, action])

    def choose_action(self, state, epsilon):
        # Epsilon-greedy policy for action selection
        if np.random.uniform() < epsilon:
            return np.random.choice(self.num_actions)
        else:
            return np.argmax(self.q_table[state])

In [ ]:
# Define the trading strategy with Q-learning and human feedback
def trading_strategy(data, q_agent, entry_points, exit_points, initial_investment):
    num_states = 2  # Number of states (0: out of the market, 1: in the market)
    num_actions = 2  # Number of actions (0: no trade, 1: trade)

    epsilon = 0.1  # Exploration rate
    total_reward = 0  # Track total reward
    portfolio_value = [initial_investment]  # Track portfolio value
    trades = []  # Track executed trades

    # Iterate over each trading day
    for i in range(len(data) - 1):
        state = 0 if portfolio_value[-1] == 0 else 1  # Current state

        # Check if an entry point exists for the current day
        if data.index[i] in entry_points:
            # Get the recommended action for the current day from human feedback
            action = entry_points[data.index[i]]

            # Execute the recommended action
            if action == 'BUY' and state == 0:
                state = 1  # Enter the market
                shares = portfolio_value[-1] / data['Close'][i]  # Buy as many shares as possible
                trades.append(('BUY', data.index[i], data['Close'][i], shares))
            elif action == 'SELL' and state == 1:
                state = 0  # Exit the market
                shares = portfolio_value[-1] / data['Close'][i]  # Sell all shares
                trades.append(('SELL', data.index[i], data['Close'][i], shares))

        # Calculate the reward based on the portfolio value change
        reward = (portfolio_value[-1] - portfolio_value[-2]) / portfolio_value[-2] if i > 0 else 0
        total_reward += reward

        # Choose the action using epsilon-greedy policy
        action = q_agent.choose_action(state, epsilon)

        # Execute the action
        if action == 1 and state == 0:
            state = 1  # Enter the market
            shares = portfolio_value[-1] / data['Close'][i]  # Buy as many shares as possible
            trades.append(('BUY', data.index[i], data['Close'][i], shares))
        elif action == 0 and state == 1:
            state = 0  # Exit the market
            shares = portfolio_value[-1] / data['Close'][i]  # Sell all shares
            trades.append(('SELL', data.index[i], data['Close'][i], shares))

        # Update the Q-table
        next_state = 0 if state == 0 else 1  # Next state
        q_agent.update_q_table(state, action, reward, next_state)

        # Update the portfolio value
        portfolio_value.append(shares * data['Close'][i])

    return total_reward, portfolio_value, trades

In [ ]:
# Define the entry and exit points (example)
entry_points = {
    '2018-04-01': 'BUY',
    '2018-05-01': 'SELL',
    '2018-08-01': 'BUY',
    '2018-09-01': 'SELL'
}

exit_points = {
    '2018-05-01': 'SELL',
    '2018-06-01': 'BUY',
    '2018-09-01': 'SELL',
    '2018-10-01': 'BUY'
}

# Set the initial investment
initial_investment = 10000

# Create the Q-learning agent
q_agent = QLearningAgent(num_states=2, num_actions=2, alpha=0.2, gamma=0.9)

# Apply the trading strategy
total_reward, portfolio_value, trades = trading_strategy(data, q_agent, entry_points, exit_points, initial_investment)


In [ ]:
# Calculate metrics
returns = np.diff(portfolio_value) / portfolio_value[:-1]
sharpe_ratio = np.sqrt(252) * np.mean(returns) / np.std(returns)
cagr = (portfolio_value[-1] / portfolio_value[0]) ** (252 / len(data)) - 1
cumulative_returns = (portfolio_value[-1] - portfolio_value[0]) / portfolio_value[0]
variance = np.var(returns)
cvar = np.mean(returns[returns < np.percentile(returns, 5)])
alpha, beta = np.polyfit(data['Close'].pct_change().dropna(), returns, deg=1)

# Plot portfolio performance
plt.figure(figsize=(10, 6))
plt.plot(data.index, portfolio_value, label='Portfolio Value')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Portfolio Performance')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Print metrics
print('Metrics:')
print(f'Sharpe Ratio: {sharpe_ratio:.3f}')
print(f'CAGR: {cagr:.3f}')
print(f'Cumulative Returns: {cumulative_returns:.3f}')
print(f'Variance: {variance:.6f}')
print(f'CVaR (Conditional Value at Risk): {cvar:.6f}')
print(f'Alpha: {alpha:.6f}')
print(f'Beta: {beta:.6f}')
print(f'portfolio_value: {portfolio_value[-1]:.6f}')

## Gym-AnyTrading Environment

-  OpenAI Gym environments for reinforcement learning-based trading algorithms.
-  Implementing three Gym environments: TradingEnv, ForexEnv, and StocksEnv.
    - TradingEnv is an abstract environment which is defined to support all kinds of trading environments. 
    - ForexEnv and StocksEnv are simply two environments that inherit and extend TradingEnv. 

- Methods (the usual for env):
    - seed: Typical Gym seed method.
    - reset: Typical Gym reset method.
    - step: Typical Gym step method.
    - render: Typical Gym render method. Renders the information of the environment's current tick.
    - render_all: Renders the whole environment.
    - close: Typical Gym close method.
______________

- The agent train on Sell=0 and Buy = 1 actions
- Long position wants to buy shares when prices are low and profit by selling them while their value is going up
- Short position wants to sell shares with high value and use this value to buy shares at a lower value, keeping the difference as profit.
    - Short = 0, Long = 1
    
_________________

- StocksEnv Properties:
    - frame_bound: A tuple which specifies the start and end of df. It is passed in the class' constructor.
    - trade_fee_bid_percent: A default constant fee percentage for bids. For example with trade_fee_bid_percent=0.01, you will lose 1% of your money every time you sell your shares.
    - trade_fee_ask_percent: A default constant fee percentage for asks. For example with trade_fee_ask_percent=0.005, you will lose 0.5% of your money every time you buy some shares.

In [ ]:
#!pip install gym-anytrading

In [ ]:
import gym
import gymnasium as gym
import gym_anytrading

#env = gym.make('forex-v0')
env = gym.make('stocks-v0')

In [ ]:
### create environment

from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL

custom_env = gym.make(
     'stocks-v0',
     df=STOCKS_GOOGL,
     window_size=10,
     frame_bound=(10, 300)
 )

In [ ]:
print("env information:")
print("> shape:", env.unwrapped.shape)
print("> df.shape:", env.unwrapped.df.shape)
print("> prices.shape:", env.unwrapped.prices.shape)
print("> signal_features.shape:", env.unwrapped.signal_features.shape)
print("> max_possible_profit:", env.unwrapped.max_possible_profit())

print()
print("custom_env information:")
print("> shape:", custom_env.unwrapped.shape)
print("> df.shape:", custom_env.unwrapped.df.shape)
print("> prices.shape:", custom_env.unwrapped.prices.shape)
print("> signal_features.shape:", custom_env.unwrapped.signal_features.shape)
print("> max_possible_profit:", custom_env.unwrapped.max_possible_profit())

In [ ]:
## Here max_possible_profit signifies that if the market didn't have trade fees, 
## you could have earned 5.1919 units of currency by starting with 1.0. 
## In other words, your money is almost quadrupled.

In [ ]:
### plot the environment

env.reset()
env.render()

In [ ]:
## Short and Long positions are shown in red and green colors.
## As you see, the starting position of the environment is always Short.

In [ ]:
### a complete example

import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL


env = gym.make('forex-v0', frame_bound=(50, 100), window_size=10)
# env = gym.make('stocks-v0', frame_bound=(50, 100), window_size=10)

observation = env.reset(seed=2023)
while True:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # env.render()
    if done:
        print("info:", info)
        break

plt.cla()
env.unwrapped.render_all()
plt.show()

- Use render_all method to avoid rendering on each step and prevent time-wasting.
- The first 10 points (window_size=10) on the plot don't have a position. 
- They aren't involved in calculating reward, profit, etc. 
- They just display the first observations. 
- So the environment's _start_tick and initial _last_trade_tick are 10 and 9.

__________________


- To get more you can use gym_anytrading with other libraries, e.g., stable_baselines3 and quantstats

In [ ]:
#!pip install stable_baselines3

In [ ]:
#!pip install quantstats

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import gymnasium as gym
import gym_anytrading
from gym_anytrading.envs import Actions

from stable_baselines3 import A2C

import quantstats as qs

In [ ]:
### create env
df = gym_anytrading.datasets.STOCKS_GOOGL.copy()

window_size = 10
start_index = window_size
end_index = len(df)

env = gym.make(
    'stocks-v0',
    df=df,
    window_size=window_size,
    frame_bound=(start_index, end_index)
)

print("observation_space:", env.observation_space)

In [ ]:
## train env (A2C algorithm)

env.reset(seed=2023)
model = A2C('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=1_000)


In [ ]:
### test env

action_stats = {Actions.Sell: 0, Actions.Buy: 0}

observation, info = env.reset(seed=2023)

while True:
    # action = env.action_space.sample()
    action, _states = model.predict(observation)

    action_stats[Actions(action)] += 1
    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

    # env.render()
    if done:
        break

env.close()

print("action_stats:", action_stats)
print("info:", info)

In [ ]:
### plot  results

plt.figure(figsize=(10, 6))
env.unwrapped.render_all()
plt.show()

In [ ]:
### backtesting results

qs.extend_pandas()

net_worth = pd.Series(env.unwrapped.history['total_profit'], index=df.index[start_index+1:end_index])
returns = net_worth.pct_change().iloc[1:]

qs.reports.full(returns)
qs.reports.html(returns, output='SB3_a2c_quantstats.html')